<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [5]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip 

--2021-04-13 13:30:30--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  19.8MB/s    in 14s     

2021-04-13 13:30:45 (18.2 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [ ]:
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [7]:
# import the libraries
import keras
from keras.applications.vgg16 import preprocess_input
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense

In [8]:
batch_size = 100
image_resize = 224

In [9]:
# list subfolders
!ls concrete_data_week4

test  train  valid


In [10]:
# create a ImageDataGenerator for training

data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = data_generator.flow_from_directory(
                        'concrete_data_week4/train',
                        target_size=(image_resize, image_resize),
                        batch_size=batch_size,
                        class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [11]:
valid_generator = data_generator.flow_from_directory(
                        'concrete_data_week4/valid',
                        target_size=(image_resize, image_resize),
                        batch_size=batch_size,
                        class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [12]:
# create the Sequential model
model = Sequential()

In [13]:
# add VGG16 to the model
model.add(VGG16(include_top=False,
                pooling='avg',
                weights='imagenet',
         ))

58892288/58889256 [==============================] - 1s 0us/step


In [14]:
# add output layer with 2 classes
model.add(Dense(2, activation='softmax'))

In [15]:
# check the layers
print(model.layers)

# num of layers of VGG16
len(model.layers[0].layers)

[<tensorflow.python.keras.engine.functional.Functional object at 0x7f8770cb2710>, <tensorflow.python.keras.layers.core.Dense object at 0x7f87709a6e50>]


20

In [16]:
# set  VGG16 as not trainable
model.layers[0].trainable = False

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [18]:
# compile the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [19]:
# set some variables
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(valid_generator)
num_epochs = 2

In [20]:
# fit the model using train data
model.fit_generator(train_generator,
                   steps_per_epoch=steps_per_epoch_training,
                   epochs=num_epochs,
                   validation_data=valid_generator,
                   validation_steps=steps_per_epoch_validation,
                   verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
301/301 [==============================] - 371s 1s/step - loss: 0.2120 - accuracy: 0.9100 - val_loss: 0.0243 - val_accuracy: 0.9947
Epoch 2/2
301/301 [==============================] - 312s 1s/step - loss: 0.0207 - accuracy: 0.9955 - val_loss: 0.0146 - val_accuracy: 0.9960


In [21]:
model.save('classifier_vgg16_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [24]:
# load the already trained resnet model
model_resnet = keras.models.load_model('classifier_resnet_model.h5')

In [25]:
# create ImageDataGenerator for the test set
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = data_generator.flow_from_directory(
                        'concrete_data_week4/test',
                        target_size=(image_resize, image_resize),
                        batch_size=batch_size,
                        class_mode='categorical',
                        # default is shuffle=True
                        shuffle=False)

Found 500 images belonging to 2 classes.


In [27]:
# evaluate resnet
results_resnet = model_resnet.evaluate(test_generator,
                                      return_dict=True)

5/5 [==============================] - 4s 690ms/step - loss: 0.0015 - accuracy: 1.0000


In [28]:
# print the results for resnet
results_resnet

{'accuracy': 1.0, 'loss': 0.0015223603695631027}

In [29]:
# evaluate VGG16
results_vgg16 = model.evaluate(test_generator,
                               return_dict=True)

5/5 [==============================] - 4s 805ms/step - loss: 0.0113 - accuracy: 0.9980


In [30]:
# print the results for VGG16
results_vgg16

{'accuracy': 0.9980000257492065, 'loss': 0.011345524340867996}

<a id="item44"></a>

## Part 3

In this part, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>

Use the following cells to make your predictions.

In [36]:
# calculate predictions with both models
resnet_pred = model_resnet.predict(test_generator)
vgg16_pred = model.predict(test_generator)

In [37]:
# check output type and length
print('type: ', type(resnet_pred))
print('length: ', len(resnet_pred))

type:  <class 'numpy.ndarray'>
length:  500


In [46]:
for out in vgg16_pred[:5]:
  if out.argmax() == 0:
    print('Positive')
  else:
    print('Negative')

Positive
Positive
Positive
Positive
Positive


In [42]:
for out in resnet_pred[:5]:
  if out.argmax() == 0:
    print('Positive')
  else:
    print('Negative')

Positive
Positive
Positive
Positive
Positive


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).